In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
!pip install pingouin
import pingouin as pg
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
data = pd.read_csv("all_seasons.csv")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
standard = ["reb", "ast", "net_rating","player_height","player_weight"]

data["gp"] = data["gp"]/82
data["draft_round"] = data["draft_round"].replace({"Undrafted":8}).astype("float")/8


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree

In [ ]:
linear = LinearRegression()
input = ["reb", "ast", "net_rating","player_height","player_weight","gp","draft_round","oreb_pct","dreb_pct", "data_age"]

In [ ]:
data["Season#"] = data["season"].str.split("-", expand = True)[0].astype(int)

In [ ]:
data

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,Season#
0,0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,0.250,...,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97,1996
1,1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,0.250,...,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97,1996
2,2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,0.125,...,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97,1996
3,3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,0.125,...,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97,1996
4,4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,0.125,...,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12839,12839,Joel Embiid,PHI,29.0,213.36,127.005760,Kansas,Cameroon,2014,0.125,...,10.2,4.2,8.8,0.057,0.243,0.370,0.655,0.233,2022-23,2022
12840,12840,John Butler Jr.,POR,20.0,213.36,86.182480,Florida State,USA,Undrafted,1.000,...,0.9,0.6,-16.1,0.012,0.065,0.102,0.411,0.066,2022-23,2022
12841,12841,John Collins,ATL,25.0,205.74,102.511792,Wake Forest,USA,2017,0.125,...,6.5,1.2,-0.2,0.035,0.180,0.168,0.593,0.052,2022-23,2022
12842,12842,Jericho Sims,NYK,24.0,208.28,113.398000,Texas,USA,2021,0.250,...,4.7,0.5,-6.7,0.117,0.175,0.074,0.780,0.044,2022-23,2022


In [ ]:
def predictor(season, model):

  current_season = season
  testing_data = data[data["Season#"] == current_season].copy(deep = True)
  training_data  = data[data["Season#"] < current_season].copy(deep = True)

  training_data["data_age"] = season - training_data["Season#"]
  testing_data["data_age"] = 0

  scaler.fit(training_data[standard])
  training_data[standard] = scaler.transform(training_data[standard])



  testing_data[standard] = scaler.transform(testing_data[standard])
  training_x = training_data[input]
  training_y = training_data["pts"]
  testing_x = testing_data[input]
  testing_y = testing_data["pts"]
  model = model.fit(training_x,training_y)
  return model.score(testing_x, testing_y)

In [ ]:
scores = []

scoring_model = SVR()

for i in range(2022-1998):
  score = predictor(i+1997,SVR())
  scores.append({"Score": score, "Date": i+1997, "Model": "SVR"})
  score = predictor(i+1997,LinearRegression())
  scores.append({"Score": score, "Date": i+1997, "Model": "LR"})
  score = predictor(i+1997,RandomForestRegressor())
  scores.append({"Score": score, "Date": i+1997, "Model": "RFR"})
  score = predictor(i+1997,DecisionTreeRegressor())
  scores.append({"Score": score, "Date": i+1997, "Model": "DTR"})

In [ ]:
scores = pd.DataFrame(scores).sort_values(by = "Model")


In [ ]:
scores

,Score,Date,Model
47,0.710629,2008,DTR
27,0.671191,2003,DTR
31,0.735713,2004,DTR
35,0.744444,2005,DTR
39,0.737016,2006,DTR
...,...,...,...
8,0.667627,1999,SVR
64,0.676510,2013,SVR
68,0.658697,2014,SVR
76,0.622317,2016,SVR


In [ ]:
mean_scores = scores.groupby("Model").mean().sort_values("Score",ascending = False).reset_index()
mean_scores["Error"] = scores.groupby("Model").sem().reset_index()["Score"]
fig1 = px.bar(mean_scores, x = "Model", y = "Score", error_y = "Error", width =500)

fig1.show()


In [ ]:
svr = SVR()
current_season = 2022

testing_data = data[data["Season#"] == current_season].copy(deep = True)
training_data  = data[data["Season#"] < current_season].copy(deep = True)

training_data["data_age"] = current_season - training_data["Season#"]
testing_data["data_age"] = 0
scaler.fit(training_data[standard])
training_data[standard] = scaler.transform(training_data[standard])
testing_data[standard] = scaler.transform(testing_data[standard])
training_x = training_data[input]
training_y = training_data["pts"]
testing_x = testing_data[input]
testing_y = testing_data["pts"]
model = svr.fit(training_x,training_y)

In [ ]:
predictions = svr.predict(testing_x)

In [ ]:
px.scatter(x=predictions, y=testing_y)

In [ ]:
testing_data["predicted pts"] = predictions
sort = "college"
px.bar(testing_data.groupby(sort, as_index = False).mean().sort_values("predicted pts",ascending = False), y = "predicted pts", x = sort)

In [ ]:
scores.groupby("Model").agg(["mean","sem"]).reset_index()[["Model","Score"]]

Model     Score          
             mean       sem
0   DTR  0.666535  0.013175
1    LR  0.731110  0.003928
2   RFR  0.834676  0.004876
3   SVR  0.675694  0.005704

In [ ]:
pg.pairwise_tests(scores, dv = "Score", within = "Model", subject = "Date")

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,Model,DTR,LR,True,True,-5.014666,23.0,two-sided,4.497827e-05,555.377,-1.333644
1,Model,DTR,RFR,True,True,-17.047885,23.0,two-sided,1.522623e-14,3.965e+11,-3.398362
2,Model,DTR,SVR,True,True,-0.710175,23.0,two-sided,4.847347e-01,0.27,-0.181152
3,Model,LR,RFR,True,True,-23.665188,23.0,two-sided,1.198713e-17,3.566e+14,-4.696566
4,Model,LR,SVR,True,True,11.278172,23.0,two-sided,7.544296e-11,1.26e+08,2.272008
5,Model,RFR,SVR,True,True,26.520061,23.0,two-sided,9.543774e-19,3.981e+15,6.015777


In [ ]:
scores

,Score,Date,Model
47,0.710629,2008,DTR
27,0.671191,2003,DTR
31,0.735713,2004,DTR
35,0.744444,2005,DTR
39,0.737016,2006,DTR
...,...,...,...
8,0.667627,1999,SVR
64,0.676510,2013,SVR
68,0.658697,2014,SVR
76,0.622317,2016,SVR


In [ ]:
from sklearn.inspection import permutation_importance
lG = RandomForestRegressor()
current_season = 2022



testing_data = data[data["Season#"] == current_season].copy(deep = True)
training_data  = data[data["Season#"] < current_season].copy(deep = True)

training_data["data_age"] = current_season - training_data["Season#"]
testing_data["data_age"] = 0
scaler.fit(training_data[standard])
training_data[standard] = scaler.transform(training_data[standard])
testing_data[standard] = scaler.transform(testing_data[standard])
training_x = training_data[input]
training_y = training_data["pts"]
testing_x = testing_data[input]
testing_y = testing_data["pts"]
model = lG.fit(training_x,training_y)
permResult  = permutation_importance(lG,training_x,training_y)

px.bar(x = input, y = permResult["importances_mean"], error_y = permResult["importances_std"])

In [ ]:

px.bar(x = input, y = permResult["importances_mean"], error_y = permResult["importances_std"])